## import packages

In [1]:
import requests
import pandas as pd
import time

In [ ]:
api_key = 'replace api key here'

## 1. text search api

In [3]:
def call_text_search_api(fieldmask, textquery, api_key, page_size=20, max_pages=3):

    url = 'https://places.googleapis.com/v1/places:searchText'

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    all_places = []
    next_page_token = None
    total_results = 0

    for page in range(max_pages):
        if next_page_token:
            params = {'nextPageToken': next_page_token}
            response = requests.get(url, headers=headers, params=params)
        else:
            data = {
                'textQuery': textquery,
                'pageSize': page_size
            }
            response = requests.post(url, headers=headers, json=data)   # post accepts a json request body which allows for more parameters to be specified
            
        result = response.json()

        num_results_page = len(result.get('places', []))
        total_results += num_results_page
        print(f"page {page + 1}: returned {num_results_page} results (total: {total_results})")

        if 'places' in result:
            all_places.extend(result['places'])

        next_page_token = result.get('nextPageToken')
        if not next_page_token:
            break

        time.sleep(3)

    df = pd.json_normalize(all_places) if all_places else pd.DataFrame()

    return df

### 1.1. searching for hawker stalls in newton food centre

In [11]:
fieldmask_1_1a = '*'   # retrieves all available fields
textquery_1_1a = 'hawker stalls in Newton Food Centre Singapore'

test_1_1a = call_text_search_api(fieldmask_1_1a, textquery_1_1a, api_key)

page 1: returned 11 results (total: 11)


In [12]:
test_1_1a

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,internationalPhoneNumber,reservable,servesDessert,regularOpeningHours.nextOpenTime,currentOpeningHours.nextOpenTime,servesBeer,servesWine,outdoorSeating,servesCoffee,servesBreakfast
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJdadh-esZ2jERLMyRFjaOLsI,ChIJdadh-esZ2jERLMyRFjaOLsI,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-51 Newton Food Centr...","[{'longText': '01-51', 'shortText': '01-51', '...",2.2,https://maps.google.com/?cid=13992277455244741...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,places/ChIJJczvxGIZ2jER4wtULz2lCbg,ChIJJczvxGIZ2jER4wtULz2lCbg,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-13 Newton Food Centr...","[{'longText': '01-13', 'shortText': '01-13', '...",3.7,https://maps.google.com/?cid=13261312259974761...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,places/ChIJu0Rc9-sZ2jERF49RylTbSEs,ChIJu0Rc9-sZ2jERF49RylTbSEs,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-25 Newton Food Centr...","[{'longText': '01-25', 'shortText': '01-25', '...",3.6,https://maps.google.com/?cid=54248269083858409...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,places/ChIJx3TNyV8Z2jERAFrgrjj3HVA,ChIJx3TNyV8Z2jERAFrgrjj3HVA,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-31 Food Centre, Sing...","[{'longText': '#01-31', 'shortText': '#01-31',...",4.4,https://maps.google.com/?cid=57730421201830118...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,+65 8224 9929,False,False,2025-11-06T04:00:00Z,2025-11-06T04:00:00Z,NaN,NaN,NaN,NaN,NaN
5,places/ChIJu3wK9-sZ2jERz3jrVfP4sWM,ChIJu3wK9-sZ2jERz3jrVfP4sWM,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-27 Newton Food Centr...","[{'longText': '#01-27', 'shortText': '#01-27',...",3.6,https://maps.google.com/?cid=71837966046348883...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,+65 9771 9209,NaN,NaN,2025-11-06T03:30:00Z,2025-11-06T03:30:00Z,True,False,True,NaN,NaN
6,places/ChIJxcbG7-sZ2jERiyhLwTAA5i8,ChIJxcbG7-sZ2jERiyhLwTAA5i8,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-19 Newton Food Centr...","[{'longText': '01-19', 'shortText': '01-19', '...",4.7,https://maps.google.com/?cid=34514463738274223...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,2025-11-06T08:30:00Z,2025-11-06T08:30:00Z,NaN,NaN,NaN,NaN,NaN
7,places/ChIJHaje8OsZ2jERp5RIkq3p2iU,ChIJHaje8OsZ2jERp5RIkq3p2iU,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-05 Newton Food Centr...","[{'longText': '01-05', 'shortText': '01-05', '...",3.0,https://maps.google.com/?cid=27277494560191008...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,places/ChIJ5SNx8OsZ2jERzcZTgn4OBSs,ChIJ5SNx8OsZ2jERzcZTgn4OBSs,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-11, Singapore 229495","[{'longText': '01-11', 'shortText': '01-11', '...",4.3,https://maps.google.com/?cid=30998998550296511...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,+65 9452 6293,NaN,NaN,2025-11-06T08:00:00Z,2025-11-06T08:00:00Z,NaN,NaN,True,False,NaN
9,places/ChIJU_PRvacZ2jERC2QJ

In [52]:
# all available fields returned
print(test_1_1a.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [23]:
# select columns of interest
test_1_1a[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
1,Newton 51.,ChIJdadh-esZ2jERLMyRFjaOLsI,restaurant,"500 Clemenceau Ave N, #01-51 Newton Food Centr...",1.312100,103.839363,2.2,[{'name': 'places/ChIJdadh-esZ2jERLMyRFjaOLsI/...,62,"[Monday: 11:00 AM – 11:00 PM, Tuesday: 11:00 A...",1,10,NaN
2,Kwee Heng,ChIJJczvxGIZ2jER4wtULz2lCbg,restaurant,"500 Clemenceau Ave N, #01-13 Newton Food Centr...",1.311854,103.839706,3.7,[{'name': 'places/ChIJJczvxGIZ2jER4wtULz2lCbg/...,93,"[Monday: 9:00 AM – 9:00 PM, Tuesday: 9:00 AM –...",1,10,NaN
3,Newton Old Signboard 25,ChIJu0Rc9-sZ2jERF49RylTbSEs,restaurant,"500 Clemenceau Ave N, #01-25 Newton Food Centr...",1.312115,103.839389,3.6,[{'name': 'places/ChIJu0Rc9-sZ2jERF49RylTbSEs/...,68,"[Monday: Closed, Tuesday: 10:45 AM – 10:30 PM,...",1,10,NaN
4,31 Heng Heng BBQ 兴兴烧烤,ChIJx3TNyV8Z2jERAFrgrjj3HVA,restaurant,"500 Clemenceau Ave N, #01-31 Food Centre, Sing...",1.311950,103.839577,4.4,[{'name': 'places/ChIJx3TNyV8Z2jERAFrgrjj3HVA/...,111,"[Monday: 12:00 – 10:30 PM, Tuesday: Closed, We...",NaN,NaN,NaN
5,Alliance Seafood,ChIJu3wK9-sZ2jERz3jrVfP4sWM,restaurant,"500 Clemenceau Ave N, #01-27 Newton Food Centr...",1.311986,103.839729,3.6,[{'name': 'places/ChIJu3wK9-sZ2jERz3jrVfP4sWM/...,188,"[Monday: Closed, Tuesday: 11:30 AM – 10:30 PM,...",NaN,NaN,NaN
6,Stingray Goh Chew Kee Seafood,ChIJxcbG7-sZ2jERiyhLwTAA5i8,restaurant,"500 Clemenceau Ave N, #01-19 Newton Food Centr...",1.311868,103.839835,4.7,[{'name': 'places/ChIJxcbG7-sZ2jERiyhLwTAA5i8/...,41,"[Monday: 4:30 – 11:30 PM, Tuesday: 4:30 – 11:3...",10,20,NaN
7,88 San Ren Cold & Hot Dessert,ChIJHaje8OsZ2jERp5RIkq3p2iU,restaurant,"500 Clemenceau Ave N, #01-05 Newton Food Centr...",1.311842,103.839394,3.0,[{'name': 'places/ChIJHaje8OsZ2jERp5RIkq3p2iU/...,37,"[Monday: 11:00 AM – 10:30 PM, Tuesday: 11:00 A...",1,10,NaN
8,Hai Yan BBQ Seafood,ChIJ5SNx8OsZ2jERzcZTgn4OBSs,restaurant,"500 Clemenceau Ave N, #01-11, Singapore 229495",1.311851,103.839553,4.3,[{'name': 'places/ChIJ5SNx8OsZ2jERzcZTgn4OBSs/...,62,"[Monday: Closed, Tuesday: 4:00 – 11:00 PM, Wed...",10,20,NaN
9,Hup Kee Fried Oyster Omelette,ChIJU_PRvacZ2jERC2QJ764xZRk,restaurant,"500 Clemenceau Ave N, Newton Circus, #01-73 Fo...",1.312241,103.839583,4.0,[{'name': 'places/ChIJU_PRvacZ2jERC2QJ764xZRk/...,111,"[Monday: Closed, Tuesday: Closed, Wednesday: 5...",1,10,NaN


In [15]:
fieldmask_1_1b = '*'
textquery_1_1b = 'hawker stalls in newton food centre singapore'   # search text is case sensitive

test_1_1b = call_text_search_api(fieldmask_1_1b, textquery_1_1b, api_key)

page 1: returned 1 results (total: 1)


In [16]:
test_1_1b

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,10,SGD,30,Asia/Singapore,SG,en-US,229495,[500 Clemenceau Ave N]


In [17]:
print(test_1_1b.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [24]:
test_1_1b[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...


In [38]:
fieldmask_1_1c = '*'
textquery_1_1c = 'stalls in Newton Food Centre Singapore'

test_1_1c = call_text_search_api(fieldmask_1_1c, textquery_1_1c, api_key)

page 1: returned 1 results (total: 1)


In [39]:
test_1_1c

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,10,SGD,30,Asia/Singapore,SG,en-US,229495,[500 Clemenceau Ave N]


In [40]:
print(test_1_1c.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [41]:
test_1_1c[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...


### 1.2. searching for a specific hawker stall in newton food centre

In [25]:
fieldmask_1_2 = '*'
textquery_1_2 = 'kwang kee teochew fish porridge in newton food centre'

test_1_2 = call_text_search_api(fieldmask_1_2, textquery_1_2, api_key)

page 1: returned 1 results (total: 1)


In [26]:
test_1_2

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,utcOffsetMinutes,adrFormatAddress,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY,ChIJPbO-7-sZ2jER1zjzFT3nUyY,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-20, Singapore 229495","[{'longText': '#01-20', 'shortText': '#01-20',...",4,https://maps.google.com/?cid=27618052460555286...,https://food.grab.com/sg/en/restaurant/kwang-k...,480,"<span class=""street-address"">500 Clemenceau Av...",...,https://www.google.com/maps/place//data=!4m3!3...,SGD,1,SGD,10,Asia/Singapore,SG,en-US,229495,"[500 Clemenceau Ave N, #01-20]"


In [27]:
print(test_1_2.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'websiteUri', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'takeout', 'dineIn',
       'servesBreakfast', 'servesLunch', 'servesDinner', 'servesBeer',
       'servesWine', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'liveMusic', 'menuForChildren', 'goodForChildren',
       'allowsDogs', 'restroom', 'goodForGroups', 'goodForWatchingSports',
       'containingPlaces', 'plusCode.globalCode', 'plusCode.compoundCode',
       'location.latitude', 'location.longitude', 'viewport.low.latitude',
       'viewport.low.longitude', 'viewport.high.latitude',
       'viewport.high.longitude', 'regularOpeningHours.openNow',
       'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions',
       'regularOpeningHours.nextCloseTime', 'displayName.text',
       'display

In [28]:
test_1_2[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units
0,Kwang Kee Teochew Fish Porridge,ChIJPbO-7-sZ2jER1zjzFT3nUyY,restaurant,"500 Clemenceau Ave N, #01-20, Singapore 229495",1.311873,103.839863,4,[{'name': 'places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/...,74,"[Monday: Closed, Tuesday: 11:00 AM – 8:00 PM, ...",1,10


In [53]:
# extract reviews (only max 5 returned)
test_1_2['reviews'][0]

[{'name': 'places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/reviews/Ci9DQUlRQUNvZENodHljRjlvT25KWWVYWjZXRGxYVDNjdFRUQkdlbEZ5TVZwWFgxRRAB',
  'relativePublishTimeDescription': '2 months ago',
  'rating': 4,
  'text': {'text': 'Food is decent but for the accolades that they had displayed (Michelin listed from 2021-2025), one would have higher expectations.\n\nI have eaten from the store in the past and I feel like their fish portions have been getting smaller. Undoubtedly, the fish was fresh and the meat was sweet. However, for $6, only five to six thin, bite-sized slices of fish were given. It’s a different case if the fish was chunky or of a thicker cut.\n\nSoup broth is light, leans towards to sweeter side.',
   'languageCode': 'en'},
  'originalText': {'text': 'Food is decent but for the accolades that they had displayed (Michelin listed from 2021-2025), one would have higher expectations.\n\nI have eaten from the store in the past and I feel like their fish portions have been getting smaller. Und

### 1.3. searching for hawker stalls in singapore

In [44]:
fieldmask_1_3 = '*'
textquery_1_3 = 'hawker stalls in singapore'

test_1_3 = call_text_search_api(fieldmask_1_3, textquery_1_3, api_key)

page 1: returned 20 results (total: 20)
page 2: returned 0 results (total: 20)


In [45]:
test_1_3   # only returns 20 entries

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,...,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines,curbsidePickup,paymentOptions.acceptsCreditCards,parkingOptions.freeParkingLot,parkingOptions.freeStreetParking,accessibilityOptions.wheelchairAccessibleParking,containingPlaces
0,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84,ChIJseQsTQ0Z2jERqpBTWF0Zf84,"[tourist_attraction, restaurant, food, point_o...",6225 5632,+65 6225 5632,"1 Kadayanallur St, Singapore 069184","[{'longText': '1', 'shortText': '1', 'types': ...",4.4,https://maps.google.com/?cid=14879639582559932...,https://www.nea.gov.sg/our-services/hawker-man...,...,SG,en-US,069184,[1 Kadayanallur St],NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,"[tourist_attraction, restaurant, food, point_o...",6220 2138,+65 6220 2138,"18 Raffles Quay, Singapore 048582","[{'longText': '18', 'shortText': '18', 'types'...",4.4,https://maps.google.com/?cid=18296669017417104...,https://laupasat.sg/,...,SG,en-US,048582,[18 Raffles Quay],False,True,False,False,False,NaN
2,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...",NaN,NaN,"500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,NaN,...,SG,en-US,229495,[500 Clemenceau Ave N],False,True,NaN,False,True,NaN
3,places/ChIJWzTvXEcY2jERvSzB8XNKKCk,ChIJWzTvXEcY2jERvSzB8XNKKCk,"[bakery, restaurant, food_store, food, point_o...",NaN,NaN,"51 Old Airport Rd, Singapore 390051","[{'longText': '51', 'shortText': '51', 'types'...",4.3,https://maps.google.com/?cid=29657022164611350...,https://www.nea.gov.sg/our-services/hawker-man...,...,SG,en-US,390051,[51 Old Airport Rd],NaN,False,False,False,True,NaN
4,places/ChIJ04DTdbQZ2jERFt4kBQi-E60,ChIJ04DTdbQZ2jERFt4kBQi-E60,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"505 Beach Rd, Singapore 199583","[{'longText': '505', 'shortText': '505', 'type...",4.3,https://maps.google.com/?cid=12471520734774353...,NaN,...,SG,en-US,199583,[505 Beach Rd],NaN,False,False,NaN,True,NaN
5,places/ChIJ14tfr0UW2jERvfZClMtLrsQ,ChIJ14tfr0UW2jERvfZClMtLrsQ,"[restaurant, food, point_of_interest, establis...",6225 5632,+65 6225 5632,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...","[{'longText': 'Ci Yuan Community Centre', 'sho...",4.2,https://maps.google.com/?cid=14172348415119390...,http://ciyuanhawker.com.sg/,...,SG,en-US,538776,"[51 Hougang Ave 9, Ci Yuan Community Centre]",NaN,False,NaN,NaN,True,NaN
6,places/ChIJOU6y-HkZ2jERchOICIgjl0E,ChIJOU6y-HkZ2jERchOICIgjl0E,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"30 Seng Poh Rd, Singapore 168898","[{'longText': '30', 'shortText': '30', 'types'...",4.3,https://maps.google.com/?cid=47262854011142849...,NaN,...,SG,en-US,168898,[30 Seng Poh Rd],NaN,False,NaN,NaN,True,NaN
7,places/ChIJ1ZbzmxIZ2jERLxbskKJNx8Q,ChIJ1ZbzmxIZ2jERLxbskKJNx8Q,"[restaurant, food, point_of_interest, establis...",6225 5632,+65 6225 5632,"7 Maxwell Rd, Singapore 069111","[{'longText': '7', 'shortText': '7', 'types': ...",4.3,https://maps.google.com/?cid=14179387312410727...,http://www.nea.gov.sg/,...,SG,en-US,069111,[7 Maxwell Rd],NaN,False,False,False,True,NaN
8,places/ChIJv4Mk4QYa2jERlGncaOIvpW4,ChIJv4Mk4QYa2jERlGncaOIvpW4,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"2 Adam Rd, Singapore 289876","[{'longText': '2', 'shortText': '2', 'types': ...",4.1,https://maps.google.com/?cid=79728313647977209...,https://www.facebook.com/adamfoodcentre/,...,SG,en-US,289876,[2 Adam Rd],NaN,False,False,NaN,True,NaN
9,places/ChIJsUJymcwZ2jERaYAaWRthb5s,ChIJsUJymcwZ2jERaYAaWRthb5s,"[food_court, food, point_of_interest, establis...",NaN,NaN,"2 Orchard Turn, #B4-66, Singapore 238801","[{'longText': '#B4-66', 'shortText': '#B4-66',...",4.6,https://maps.google.com/?cid=11

In [46]:
print(test_1_3.columns)

Index(['name', 'id', 'types', 'nationalPhoneNumber',
       'internationalPhoneNumber', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'websiteUri', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'priceLevel', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'takeout', 'delivery',
       'dineIn', 'reservable', 'servesBreakfast', 'servesLunch',
       'servesDinner', 'servesBeer', 'servesWine', 'servesBrunch',
       'servesVegetarianFood', 'primaryType', 'shortFormattedAddress',
       'reviews', 'photos', 'outdoorSeating', 'liveMusic', 'menuForChildren',
       'servesDessert', 'servesCoffee', 'goodForChildren', 'allowsDogs',
       'restroom', 'goodForGroups', 'goodForWatchingSports',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpening

In [48]:
test_1_3[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Maxwell Food Centre,ChIJseQsTQ0Z2jERqpBTWF0Zf84,restaurant,"1 Kadayanallur St, Singapore 069184",1.280336,103.844767,4.4,[{'name': 'places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/...,21733,"[Monday: 8:00 AM – 10:00 PM, Tuesday: 8:00 AM ...",1,10,Chinatown hawker center with stands selling st...
1,Lau Pa Sat,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,restaurant,"18 Raffles Quay, Singapore 048582",1.280512,103.850381,4.4,[{'name': 'places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0/...,35480,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,20,Rebuilt Victorian covered hawker centre in CBD...
2,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
3,Old Airport Road Food Centre,ChIJWzTvXEcY2jERvSzB8XNKKCk,restaurant,"51 Old Airport Rd, Singapore 390051",1.308240,103.885844,4.3,[{'name': 'places/ChIJWzTvXEcY2jERvSzB8XNKKCk/...,12383,"[Monday: 6:00 AM – 10:30 PM, Tuesday: 6:00 AM ...",1,10,Busy indoor hawker center with stalls serving ...
4,Golden Mile Food Centre,ChIJ04DTdbQZ2jERFt4kBQi-E60,restaurant,"505 Beach Rd, Singapore 199583",1.303091,103.863930,4.3,[{'name': 'places/ChIJ04DTdbQZ2jERFt4kBQi-E60/...,7284,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",1,10,NaN
5,Ci Yuan Hawker Centre,ChIJ14tfr0UW2jERvfZClMtLrsQ,restaurant,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...",1.375086,103.882926,4.2,[{'name': 'places/ChIJ14tfr0UW2jERvfZClMtLrsQ/...,2462,"[Monday: 7:00 AM – 10:00 PM, Tuesday: 7:00 AM ...",1,10,NaN
6,Tiong Bahru Market,ChIJOU6y-HkZ2jERchOICIgjl0E,restaurant,"30 Seng Poh Rd, Singapore 168898",1.285077,103.832807,4.3,[{'name': 'places/ChIJOU6y-HkZ2jERchOICIgjl0E/...,10605,NaN,1,10,Lively food court with Singaporean hawker stal...
7,Amoy Street Food Centre,ChIJ1ZbzmxIZ2jERLxbskKJNx8Q,restaurant,"7 Maxwell Rd, Singapore 069111",1.279427,103.846796,4.3,[{'name': 'places/ChIJ1ZbzmxIZ2jERLxbskKJNx8Q/...,3471,"[Monday: 6:30 AM – 9:00 PM, Tuesday: 6:30 AM –...",1,10,Hawker center whose endless stalls offer Singa...
8,Adam Food Centre,ChIJv4Mk4QYa2jERlGncaOIvpW4,restaurant,"2 Adam Rd, Singapore 289876",1.324041,103.814155,4.1,[{'name': 'places/ChIJv4Mk4QYa2jERlGncaOIvpW4/...,7994,"[Monday: 7:00 AM – 2:00 AM, Tuesday: 7:00 AM –...",1,10,NaN
9,Hawkers' Street @ ION,ChIJsUJymcwZ2jERaYAaWRthb5s,food_court,"2 Orchard Turn, #B4-66, Singapore 238801",1.303929,103.831951,4.6,[{'name': 'places/ChIJsUJymcwZ2jERaYAaWRthb5s/...,1509,"[Monday: 10:30 AM – 9:00 PM, Tuesday: 10:30 AM...",1,10,NaN


### 1.4. searching for hawker centres in singapore

In [49]:
fieldmask_1_4 = '*'
textquery_1_4 = 'hawker centres in singapore'

test_1_4 = call_text_search_api(fieldmask_1_4, textquery_1_4, api_key)

page 1: returned 20 results (total: 20)
page 2: returned 0 results (total: 20)


In [50]:
test_1_4   # only returns 20 entries

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,...,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines,curbsidePickup,paymentOptions.acceptsCreditCards,parkingOptions.freeParkingLot,parkingOptions.freeStreetParking,accessibilityOptions.wheelchairAccessibleParking,containingPlaces,parkingOptions.valetParking
0,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84,ChIJseQsTQ0Z2jERqpBTWF0Zf84,"[tourist_attraction, restaurant, food, point_o...",6225 5632,+65 6225 5632,"1 Kadayanallur St, Singapore 069184","[{'longText': '1', 'shortText': '1', 'types': ...",4.4,https://maps.google.com/?cid=14879639582559932...,https://www.nea.gov.sg/our-services/hawker-man...,...,en-US,069184,[1 Kadayanallur St],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,"[tourist_attraction, restaurant, food, point_o...",6220 2138,+65 6220 2138,"18 Raffles Quay, Singapore 048582","[{'longText': '18', 'shortText': '18', 'types'...",4.4,https://maps.google.com/?cid=18296669017417104...,https://laupasat.sg/,...,en-US,048582,[18 Raffles Quay],False,True,False,False,False,NaN,NaN
2,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...",NaN,NaN,"500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,NaN,...,en-US,229495,[500 Clemenceau Ave N],False,True,NaN,False,True,NaN,NaN
3,places/ChIJ04DTdbQZ2jERFt4kBQi-E60,ChIJ04DTdbQZ2jERFt4kBQi-E60,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"505 Beach Rd, Singapore 199583","[{'longText': '505', 'shortText': '505', 'type...",4.3,https://maps.google.com/?cid=12471520734774353...,NaN,...,en-US,199583,[505 Beach Rd],NaN,False,False,NaN,True,NaN,NaN
4,places/ChIJWzTvXEcY2jERvSzB8XNKKCk,ChIJWzTvXEcY2jERvSzB8XNKKCk,"[bakery, food_store, restaurant, food, point_o...",NaN,NaN,"51 Old Airport Rd, Singapore 390051","[{'longText': '51', 'shortText': '51', 'types'...",4.3,https://maps.google.com/?cid=29657022164611350...,https://www.nea.gov.sg/our-services/hawker-man...,...,en-US,390051,[51 Old Airport Rd],NaN,False,False,False,True,NaN,NaN
5,places/ChIJ14tfr0UW2jERvfZClMtLrsQ,ChIJ14tfr0UW2jERvfZClMtLrsQ,"[restaurant, food, point_of_interest, establis...",6225 5632,+65 6225 5632,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...","[{'longText': 'Ci Yuan Community Centre', 'sho...",4.2,https://maps.google.com/?cid=14172348415119390...,http://ciyuanhawker.com.sg/,...,en-US,538776,"[51 Hougang Ave 9, Ci Yuan Community Centre]",NaN,False,NaN,NaN,True,NaN,NaN
6,places/ChIJsUJymcwZ2jERaYAaWRthb5s,ChIJsUJymcwZ2jERaYAaWRthb5s,"[food_court, food, point_of_interest, establis...",NaN,NaN,"2 Orchard Turn, #B4-66, Singapore 238801","[{'longText': '#B4-66', 'shortText': '#B4-66',...",4.6,https://maps.google.com/?cid=11200277568380633...,https://ionorchard.com/shop/hawkers-street,...,en-US,238801,"[2 Orchard Turn, #B4-66]",NaN,NaN,NaN,NaN,True,[{'name': 'places/ChIJu6qq6Y0Z2jERZQZRLwnba5k'...,NaN
7,places/ChIJQ0ge3fEZ2jER3M6bSkGnWec,ChIJQ0ge3fEZ2jER3M6bSkGnWec,"[indian_restaurant, restaurant, food, point_of...",NaN,NaN,"665 Buffalo Road Zhujiao Centre, Tekka, Market...",[{'longText': '665 Buffalo Road Zhujiao Centre...,4.2,https://maps.google.com/?cid=16670539394555039...,NaN,...,en-US,210665,"[665 Buffalo Road Zhujiao Centre, Tekka, Market]",NaN,NaN,NaN,NaN,True,NaN,NaN
8,places/ChIJOU6y-HkZ2jERchOICIgjl0E,ChIJOU6y-HkZ2jERchOICIgjl0E,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"30 Seng Poh Rd, Singapore 168898","[{'longText': '30', 'shortText': '30', 'types'...",4.3,https://maps.google.com/?cid=47262854011142849...,NaN,...,en-US,168898,[30 Seng Poh Rd],NaN,False,NaN,NaN,True,NaN,NaN
9,places/ChIJq-d3hRY82jERfOWbep52mCs,ChIJq-d3hRY82jERfOWbep52mCs,"[food_court, restaurant, food, point_of_intere...",NaN,NaN,"2 Changi Village Rd, Singapore 50

In [51]:
print(test_1_4.columns)

Index(['name', 'id', 'types', 'nationalPhoneNumber',
       'internationalPhoneNumber', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'websiteUri', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'priceLevel', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'takeout', 'delivery',
       'dineIn', 'reservable', 'servesBreakfast', 'servesLunch',
       'servesDinner', 'servesBeer', 'servesWine', 'servesBrunch',
       'servesVegetarianFood', 'primaryType', 'shortFormattedAddress',
       'reviews', 'photos', 'outdoorSeating', 'liveMusic', 'menuForChildren',
       'servesDessert', 'servesCoffee', 'goodForChildren', 'allowsDogs',
       'restroom', 'goodForGroups', 'goodForWatchingSports',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpening

In [55]:
test_1_4[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Maxwell Food Centre,ChIJseQsTQ0Z2jERqpBTWF0Zf84,restaurant,"1 Kadayanallur St, Singapore 069184",1.280336,103.844767,4.4,[{'name': 'places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/...,21733,"[Monday: 8:00 AM – 10:00 PM, Tuesday: 8:00 AM ...",1,10,Chinatown hawker center with stands selling st...
1,Lau Pa Sat,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,restaurant,"18 Raffles Quay, Singapore 048582",1.280512,103.850381,4.4,[{'name': 'places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0/...,35480,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,20,Rebuilt Victorian covered hawker centre in CBD...
2,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
3,Golden Mile Food Centre,ChIJ04DTdbQZ2jERFt4kBQi-E60,restaurant,"505 Beach Rd, Singapore 199583",1.303091,103.863930,4.3,[{'name': 'places/ChIJ04DTdbQZ2jERFt4kBQi-E60/...,7284,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",1,10,NaN
4,Old Airport Road Food Centre,ChIJWzTvXEcY2jERvSzB8XNKKCk,restaurant,"51 Old Airport Rd, Singapore 390051",1.308240,103.885844,4.3,[{'name': 'places/ChIJWzTvXEcY2jERvSzB8XNKKCk/...,12383,"[Monday: 6:00 AM – 10:30 PM, Tuesday: 6:00 AM ...",1,10,Busy indoor hawker center with stalls serving ...
5,Ci Yuan Hawker Centre,ChIJ14tfr0UW2jERvfZClMtLrsQ,restaurant,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...",1.375086,103.882926,4.2,[{'name': 'places/ChIJ14tfr0UW2jERvfZClMtLrsQ/...,2462,"[Monday: 7:00 AM – 10:00 PM, Tuesday: 7:00 AM ...",1,10,NaN
6,Hawkers' Street @ ION,ChIJsUJymcwZ2jERaYAaWRthb5s,food_court,"2 Orchard Turn, #B4-66, Singapore 238801",1.303929,103.831951,4.6,[{'name': 'places/ChIJsUJymcwZ2jERaYAaWRthb5s/...,1509,"[Monday: 10:30 AM – 9:00 PM, Tuesday: 10:30 AM...",1,10,NaN
7,Tekka Centre,ChIJQ0ge3fEZ2jER3M6bSkGnWec,restaurant,"665 Buffalo Road Zhujiao Centre, Tekka, Market...",1.306384,103.850701,4.2,[{'name': 'places/ChIJQ0ge3fEZ2jER3M6bSkGnWec/...,22515,NaN,1,10,Mixed-use market building established in 1915 ...
8,Tiong Bahru Market,ChIJOU6y-HkZ2jERchOICIgjl0E,restaurant,"30 Seng Poh Rd, Singapore 168898",1.285077,103.832807,4.3,[{'name': 'places/ChIJOU6y-HkZ2jERchOICIgjl0E/...,10605,NaN,1,10,Lively food court with Singaporean hawker stal...
9,Changi Village Hawker Centre,ChIJq-d3hRY82jERfOWbep52mCs,restaurant,"2 Changi Village Rd, Singapore 500002",1.389237,103.988301,4.3,[{'name': 'places/ChIJq-d3hRY82jERfOWbep52mCs/...,13757,"[Monday: 6:00 AM – 2:00 AM, Tuesday: 6:00 AM –...",1,10,"Vendors selling nasi lemak rice, fried bananas..."


In [63]:
# extract individual reviews
print(test_1_4['reviews'][0][0].get('text').get('text'))   # 1st review of the 1st result of test_1_4 (maxwell food centre)
print(test_1_4['reviews'][0][1].get('text').get('text'))   # 2nd review of the 1st result of test_1_4
print(test_1_4['reviews'][0][2].get('text').get('text'))   # 3rd review of the 1st result of test_1_4
print(test_1_4['reviews'][0][3].get('text').get('text'))   # 4th review of the 1st result of test_1_4
print(test_1_4['reviews'][0][4].get('text').get('text'))   # 5th review of the 1st result of test_1_4, max number of reviews returned

Food center with very fair prices and delicious food. Most stalls also accept card payments, but it’s always good to have some cash with you. We found the place to be very clean overall — even the toilets. There’s a great variety of food, and we easily found a place to sit.
This hawker center is conveniently located on top of the MRT, offering a variety of delicious food options. The famous chicken rice is highly recommended, but it’s worth noting that other chicken rice dishes are equally tasty. There are also many refreshing drinks, such as sugarcane with lemon.

One recent addition to the menu is chilli ban mian stall, which is a delightful dish featuring eggs, anchovies, fish balls, and prawn paste balls. While it may not be overly spicy, it’s well-made and worth trying. There are similar stall that also sell equally nice noodle and some dumpling.

Overall, this hawker center is a great place to explore local cuisine, with a wide range of dishes from Western to Eastern influences. 

## 2. nearby search api

In [61]:
def call_nearby_search_api(fieldmask, loc_restriction, included_types, api_key):
    
    url = 'https://places.googleapis.com/v1/places:searchNearby'

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    data = {
        'locationRestriction': loc_restriction,
        'includedTypes': included_types
    }

    response = requests.post(url, headers=headers, json=data)
    result = response.json()

    places_list = result.get('places', [])

    df = pd.json_normalize(places_list)

    return df

In [60]:
# searching for restaurants within a 100-metre radius from newton food centre

fieldmask_2 = '*'
loc_restriction = {
  "circle": {
    "center": { "latitude": 1.3521, "longitude": 103.8198 },
    "radius": 30
  }
}
included_types = ['restaurant', 'food_court']   # 'hawker_stall' is not a supported type, need to use generic types (e.g. 'restaurant', 'cafe')

test_2 = call_nearby_search_api(fieldmask_2, loc_restriction, included_types, api_key)

NameError: name 'call_nearby_search_api' is not defined

In [149]:
test_2

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,...,servesLunch,servesDinner,servesBrunch,liveMusic,servesCoffee,goodForChildren,goodForGroups,accessibilityOptions.wheelchairAccessibleParking,servesBreakfast,accessibilityOptions.wheelchairAccessibleEntrance
0,places/ChIJFYrmYr492jERedys0dSFTDI,ChIJFYrmYr492jERedys0dSFTDI,"[vegetarian_restaurant, restaurant, food, poin...",8771 5803,+65 8771 5803,"477, #01-192, Singapore 523477","[{'longText': '477', 'languageCode': 'en'}, {'...",3.8,https://maps.google.com/?cid=36244189492237343...,https://www.instagram.com/p/DPVK2d9kz1Z/?utm_s...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJAQo3aDcR2jERRBG3b1Ak1Hs,ChIJAQo3aDcR2jERRBG3b1Ak1Hs,"[restaurant, food, point_of_interest, establis...",8833 0033,+65 8833 0033,Singapore 123456,"[{'longText': 'Singapore', 'shortText': 'Singa...",NaN,https://maps.google.com/?cid=89227966896180473...,NaN,...,True,True,True,False,True,True,True,True,NaN,NaN
2,places/ChIJBYSoMRIR2jERE2qfy1xjvdM,ChIJBYSoMRIR2jERE2qfy1xjvdM,"[indian_restaurant, restaurant, food, point_of...",NaN,NaN,"Singapore 809939921, Singapore","[{'longText': 'Singapore 809939921', 'shortTex...",NaN,https://maps.google.com/?cid=15257460362805471...,NaN,...,True,True,True,False,True,True,NaN,True,True,NaN
3,places/ChIJz7DMF1sR2jERyj0DZ_hR0DM,ChIJz7DMF1sR2jERyj0DZ_hR0DM,"[indian_restaurant, restaurant, food, point_of...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=37335742184121461...,NaN,...,True,True,True,False,True,NaN,True,NaN,NaN,NaN
4,places/ChIJgSewkZQR2jERmKbPOQmUcAE,ChIJgSewkZQR2jERmKbPOQmUcAE,"[restaurant, food, point_of_interest, establis...",6789 6789,+65 6789 6789,"13 mos, test street, Singapore 123456","[{'longText': 'test street', 'shortText': 'tes...",NaN,https://maps.google.com/?cid=10374555877504783...,NaN,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
5,places/ChIJ_VzKVgAR2jERPuiRf7izOpY,ChIJ_VzKVgAR2jERPuiRf7izOpY,"[restaurant, food, point_of_interest, establis...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=10825162259334031...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,places/ChIJFY2_SSAR2jERGBbkP5Z5Qr8,ChIJFY2_SSAR2jERGBbkP5Z5Qr8,"[fast_food_restaurant, restaurant, food, point...",NaN,NaN,"Lentor Modern, Singapore 788887","[{'longText': 'Lentor Modern', 'shortText': 'L...",NaN,https://maps.google.com/?cid=13781711495931106...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,places/ChIJGQAlHQAR2jERhCY89XPmM1I,ChIJGQAlHQAR2jERhCY89XPmM1I,"[restaurant, food, point_of_interest, establis...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=59233313206323216...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,places/ChIJjWzqWQAR2jERZJNw4-sg_Bc,ChIJjWzqWQAR2jERZJNw4-sg_Bc,"[restaurant, food, point_of_interest, establis...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=17282925545086370...,NaN,...,NaN,True,NaN,False,NaN,True,NaN,NaN,NaN,NaN
9,places/ChIJMakes-0R2jERiCtvDVOLWhg,ChIJMakes-0R2jERiCtvDVOLWhg,"[restaurant, food, point_of_interest, establis...",28 287 417,+371 28 287 417,"88 Zion Rd, Egg Harbor Twp, Singapore 160088","[{'longText': 'Egg Harbor Twp', 'shortText': '...",1.0,https://maps.google.com/?cid=17548681936381653...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
print(test_2.columns)

Index(['name', 'id', 'types', 'nationalPhoneNumber',
       'internationalPhoneNumber', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'websiteUri', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'servesVegetarianFood',
       'primaryType', 'shortFormattedAddress', 'reviews', 'photos',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions',
       'regularOpeningHours.nextCloseTime', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpe

In [147]:
test_2[['displayName.text',
        'formattedAddress',
        'primaryType',
        'rating',
        'reviews',
        'userRatingCount']]

,displayName.text,formattedAddress,primaryType,rating,reviews,userRatingCount
0,SHU Vegetarian 蔬方斋,"477, #01-192, Singapore 523477",vegetarian_restaurant,3.8,[{'name': 'places/ChIJFYrmYr492jERedys0dSFTDI/...,12.0
1,Fun Toast,Singapore 123456,restaurant,NaN,NaN,NaN
2,Utpal wary,"Singapore 809939921, Singapore",indian_restaurant,NaN,NaN,NaN
3,Teongchewkeng,Singapore,indian_restaurant,NaN,NaN,NaN
4,MOS Test,"13 mos, test street, Singapore 123456",restaurant,NaN,NaN,NaN
5,Jay man Kali ka dhaba,Singapore,restaurant,NaN,NaN,NaN
6,Burger King Lentor Modern,"Lentor Modern, Singapore 788887",fast_food_restaurant,NaN,NaN,NaN
7,18 chefs,Singapore,restaurant,NaN,NaN,NaN
8,Sushi dau,Singapore,restaurant,NaN,NaN,NaN
9,Muhamadhomesg,"88 Zion Rd, Egg Harbor Twp, Singapore 160088",restaurant,1.0,[{'name': 'places/ChIJMakes-0R2jERiCtvDVOLWhg/...,1.0


## place details api

In [116]:
def call_place_details_api(fieldmask, place_id, api_key):

    url = 'https://places.googleapis.com/v1/places/' + place_id

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    response = requests.get(url, headers=headers)   # get retrieves a single resource based on a unique id
    result = response.json()

    df = pd.json_normalize(result)

    return df

In [117]:
# newton food centre (place id retrieved from text search)

fieldmask_3 = '*'
id = 'ChIJRywE--sZ2jERFLd3yhyGxUQ'   # newton food centre

test_3 = call_place_details_api(fieldmask_2, id, api_key)

In [118]:
test_3

,name,id,types,formattedAddress,addressComponents,googleMapsUri,utcOffsetMinutes,adrFormatAddress,iconMaskBaseUri,iconBackgroundColor,...,displayName.languageCode,addressDescriptor.landmarks,addressDescriptor.areas,googleMapsLinks.directionsUri,googleMapsLinks.placeUri,googleMapsLinks.photosUri,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.addressLines
0,places/ChIJRywE--sZ2jERFLd3yhyGxUQ,ChIJRywE--sZ2jERFLd3yhyGxUQ,"[premise, street_address]","Newton, Newton Food Centre, Singapore","[{'longText': 'Newton Food Centre', 'shortText...",https://maps.google.com/?cid=49555144232051484...,480,"<span class=""street-address"">Newton</span>, Ne...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,...,en,[{'name': 'places/ChIJ6wSMeewZ2jERnRkQnVf_7-c'...,[{'name': 'places/ChIJRywE--sZ2jERFLd3yhyGxUQ'...,https://www.google.com/maps/dir//''/data=!4m7!...,https://maps.google.com/?cid=49555144232051484...,https://www.google.com/maps/place//data=!4m3!3...,Asia/Singapore,SG,en-US,[Newton]


In [119]:
print(test_3.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'iconMaskBaseUri', 'iconBackgroundColor', 'shortFormattedAddress',
       'photos', 'pureServiceAreaBusiness', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude', 'displayName.text',
       'displayName.languageCode', 'addressDescriptor.landmarks',
       'addressDescriptor.areas', 'googleMapsLinks.directionsUri',
       'googleMapsLinks.placeUri', 'googleMapsLinks.photosUri', 'timeZone.id',
       'postalAddress.regionCode', 'postalAddress.languageCode',
       'postalAddress.addressLines'],
      dtype='object')
